In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

In [3]:
#data preprocessing
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


In [6]:
#encoding categorical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [7]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [10]:
#onehot encoding for the geography column (countries)
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [11]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [12]:
#splitting the data into training and test set
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y , test_size = 0.2, random_state = 0)

In [15]:
#Feature scaling (important in deep learning)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [16]:
#Building the ANN

In [17]:
ann = tf.keras.models.Sequential()

In [19]:
#adding the first input layer
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) #relu is the rectifier (threshold, hyperbolic tangent, rectifier and sigmoid)

In [20]:
#adding the seconf input layer (its same like first)
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [21]:
#Adding the output layer
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [22]:
#Training the ANN
#Compiling the ANN
ann.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy']) #metrics is a list hence '[]'

In [23]:
#Training the ANN on the training set
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)


Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5624 - accuracy: 0.7663
Epoch 2/100
250/250 [==============================] - 0s 834us/step - loss: 0.4610 - accuracy: 0.7993
Epoch 3/100
250/250 [==============================] - 0s 807us/step - loss: 0.4379 - accuracy: 0.8044
Epoch 4/100
250/250 [==============================] - 0s 951us/step - loss: 0.4275 - accuracy: 0.8091
Epoch 5/100
250/250 [==============================] - 0s 872us/step - loss: 0.4191 - accuracy: 0.8129
Epoch 6/100
250/250 [==============================] - 0s 815us/step - loss: 0.4117 - accuracy: 0.8224
Epoch 7/100
250/250 [==============================] - 0s 778us/step - loss: 0.4038 - accuracy: 0.8298
Epoch 8/100
250/250 [==============================] - 0s 790us/step - loss: 0.3935 - accuracy: 0.8332
Epoch 9/100
250/250 [==============================] - 0s 926us/step - loss: 0.3822 - accuracy: 0.8393
Epoch 10/100
250/250 [==============================] - 0s 961us/step - los

In [30]:
print(ann.predict(sc.transform([[1,0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.03777212]]


In [32]:
#predicting the test set result
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [35]:
cm = confusion_matrix(y_pred, y_test)
print(cm)
accuracy_score(y_pred, y_test)

[[1572  312]
 [  23   93]]


0.8325